# Calculate the energy-volume curve

## Write the Input File

In [2]:
%%writefile ev_curve.in
# LAMMPS INPUT SCRIPT
# Calculate the lattice constant and cohesive energy of an fcc structure

# INITIALIZATION
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 
atom_modify map array

# ATOM DEFINITION
lattice fcc ${latconst} orient x 1 0 0 orient y 0 1 0 orient z 0 0 1  
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box
replicate 1 1 1

# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy 
pair_coeff * * CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt ${element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
# DEFINE COMPUTES 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

# MINIMIZATION
reset_timestep 0 
thermo 10 
thermo_style custom step pe lx ly lz press c_eatoms 
min_style cg 
minimize 1e-25 1e-25 5000 10000 

variable natoms equal "count(all)" 
variable teng equal "c_eatoms"
variable volume equal "lx*ly*lz"
variable energy_per_atom equal "v_teng/v_natoms"

print "Total energy (eV) = ${teng}"
print "Number of atoms = ${natoms}"
print "Volume (Angstoms^3) = ${volume}"
print "Energy per atom (eV/atom) = ${energy_per_atom}"

# OUTPUT THE FINAL RELAXED STRUCTURE
write_data relaxed_${element}_${count}.data

print "All done!"

Overwriting ev_curve.in


## Run LAMMPS

In [18]:
import os
import numpy as np
import subprocess 
import shlex
import shutil

input_file = "ev_curve.in"
potential_file = "CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt"
element = "Al"
equilibrium_lattice_constant = 4.050
equilibrium_volume = equilibrium_lattice_constant**3

strains = np.linspace(-0.1, 0.1, 11)
volumes = equilibrium_volume * (1 + strains)
lattice_constants = volumes**(1/3)

output_dir = f'ev_curve_{element}'
os.makedirs(f'ev_curve_{element}', exist_ok=True)
shutil.copy(input_file, output_dir)
shutil.copy(potential_file, output_dir)

count = 0
for lattice_constant in lattice_constants:
    command_line = f'lmp -in {input_file} -var element {element} -var latconst {lattice_constant:0.02f} -var count {count} -log log_{count}.lammps'
    args = shlex.split(command_line)
    count += 1
    p = subprocess.Popen(args, cwd=output_dir)

LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.16 4.16 4.16
Created orthogonal box = (0 0 0) to (4.16 4.16 4.16)
  1 by 1 by 1 MPI processor grid
Created 4 atoms
  using lattice units in orthogonal box = (0 0 0) to (4.16 4.16 4.16)
  create_atoms

In [19]:
import warnings
from pymatgen.io.lammps.data import LammpsData
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Suppress the specific DeprecationWarning from spglib
warnings.filterwarnings("ignore", category=DeprecationWarning, module="spglib")

# Get the list of log files
log_files = [f'ev_curve_{element}/log_{i}.lammps' for i in range(len(strains))]

number_of_atoms_list = []
volume_list = []
energy_list = []
for log_file in log_files:
    with open(log_file) as f:
        for line in f:
            if 'Number of atoms' in line and 'print' not in line:
                number_of_atoms = int(line.split()[-1])
                number_of_atoms_list.append(number_of_atoms)
            if 'Volume' in line and 'print' not in line:
                volume = float(line.split()[-1])
                volume_list.append(volume)
            if 'Total energy' in line and 'print' not in line:
                energy = float(line.split()[-1])
                energy_list.append(energy)

volume_per_atom_list = [volume/number_of_atoms for volume, number_of_atoms in zip(volume_list, number_of_atoms_list)]
energy_per_atom_list = [energy/number_of_atoms for energy, number_of_atoms in zip(energy_list, number_of_atoms_list)]

# Get the list of relaxed_Cu_*.data files
data_files = [f'ev_curve_{element}/relaxed_{element}_{i}.data' for i in range(len(strains))]

space_group_list = []
for data_file in data_files:
    lammps_data = LammpsData.from_file(data_files[0], atom_style='atomic')
    structure = lammps_data.structure
    sga = SpacegroupAnalyzer(structure)
    space_group = sga.get_space_group_symbol()
    space_group_list.append(space_group)
    
config_list = [element] * len(number_of_atoms_list)

In [20]:
import pandas as pd
import dfttk.eos_fit as eos_fit

df = pd.DataFrame(
    list(zip(config_list, number_of_atoms_list, volume_list, volume_per_atom_list, energy_list, energy_per_atom_list, space_group_list)), 
    columns=['config', 'number_of_atoms', 'volume', 'volume_per_atom', 'energy', 'energy_per_atom', 'space_group'])

In [21]:
eos_values_df, eos_parameters_df = eos_fit.fit_to_all_eos(df)
eos_parameters_df

,config,eos,number_of_atoms,a,b,c,d,e,V0,E0,B,BP,B2P
0,Al,mBM4,4,58.756800,-356.747618,-706.849761,3.859869e+03,0.000000e+00,66.459536,-14.318861,70.262338,3.442826,-0.040721
1,Al,mBM5,4,-6478.159237,105356.543762,-641675.258996,1.730812e+06,-1.744513e+06,66.435421,-14.319736,74.104490,3.857992,-0.895137
2,Al,BM4,4,27.877981,-1538.635768,16411.320591,-4.144213e+04,0.000000e+00,66.460858,-14.318877,70.316661,3.428161,-0.051823
3,Al,BM5,4,-389.536531,25754.557398,-652315.285530,7.235294e+06,-2.967103e+07,66.431424,-14.319754,74.169771,3.995477,-0.904292
4,Al,LOG4,4,772.199522,-501.024833,104.797806,-7.165090e+00,0.000000e+00,66.457300,-14.318879,70.355997,3.473125,-0.065996
5,Al,LOG5,4,-23923.484669,23077.181469,-8336.130915,1.335741e+03,-8.011078e+01,66.438154,-14.319761,74.223313,3.739888,-0.919316
6,Al,murnaghan,4,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,66.462166,-14.318828,70.106398,3.404115,0.000000
7,Al,vinet,4,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,66.458789,-14.318878,70.337016,3.454531,-0.059470
8,Al,morse,4,3483.411483,-81243.053632,471765.479739,-6.053899e-01,0.000000e+00,66.458957,-14.318876,70.327690,3.452184,-0.057028


In [22]:
fig = eos_fit.plot_ev(df, eos_name="BM4")